In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

from IPython.display import Markdown, display

%matplotlib inline


## Load Dataset

In [ ]:
# privileged and unprivileged groups
privileged_groups = [{'sex': 1, 'age_binary': 1}] # old white males
unprivileged_groups = [{'sex': 0, 'age_binary': 0}]

In [31]:
#TODO : add brief explanation in a markdown cell before this code block

# custom processing for the dataset
def custom_preprocessing(df):
    median_age = df['age'].median()
    df['age_binary'] = (df['age'] > median_age).astype(float)
    df.drop(columns=['age'], inplace=True)
    df['race'] = (df['race'] == 'White').astype(float)
    df['sex'] = (df['sex'] == 'Male').astype(float)
    return df

dataset = AdultDataset(custom_preprocessing=custom_preprocessing,
                              protected_attribute_names=['age_binary', 'sex'],
                              privileged_classes=[np.array([1.0]), np.array([1.0])] ) # old white males

# Get the dataset and split into train and test
np.random.seed(1)
dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

#### Clean up training data

In [32]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(31655, 98)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['age_binary', 'sex']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['education-num', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'age_binary', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Married-civ-spouse', 'marital-status=Married-spouse-absent', 'marital-status=Never-married', 'marital-status=Separated', 'marital-status=Widowed', 'occupation=Adm-clerical', 'occupation=Armed-Forces', 'occupation=Craft-repair', 'occupation=Exec-managerial', 'occupation=Farming-fishing', 'occupation=Handlers

In [33]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.363363


In [34]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [35]:
print('The 50 first instance weights originally:')
print(dataset.instance_weights[:50])

print('The 50 first instance weights after reweighing:')
dataset_transf_train.instance_weights[:50]

The 50 first instance weights originally:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
The 50 first instance weights after reweighing:


array([1.        , 1.33826683, 1.        , 1.        , 1.        ,
       1.        , 0.81264928, 1.        , 1.        , 1.33826683,
       1.33826683, 1.        , 1.        , 1.33826683, 1.        ,
       1.33826683, 1.        , 1.        , 1.        , 1.33826683,
       0.56633898, 1.33826683, 1.        , 0.81264928, 1.        ,
       1.        , 0.81264928, 1.        , 1.33826683, 1.33826683,
       0.81264928, 0.81264928, 0.56633898, 1.33826683, 0.81264928,
       1.33826683, 0.56633898, 0.81264928, 1.33826683, 0.56633898,
       3.31574203, 0.81264928, 1.        , 3.31574203, 1.        ,
       1.        , 1.        , 1.        , 0.56633898, 1.        ])

In [45]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.000000


## Train classifier on original/weighted data

In [37]:
# --- Extract features and labels ---
def extract_xy(dataset):
    return dataset.features, dataset.labels.ravel()

In [ ]:
X_train, y_train = extract_xy(dataset_orig_train)
X_valid, y_valid = extract_xy(dataset_orig_valid)
X_test,  y_test  = extract_xy(dataset_orig_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

X_train_transf, y_train_transf = extract_xy(dataset_transf_train)
w_train_transf = dataset_transf_train.instance_weights
X_train_transf_scaled = scaler.fit_transform(X_train_transf)

In [39]:
lr_classifier_origin = LogisticRegression(max_iter=1000)
lr_classifier_origin.fit(X_train_scaled, y_train)
y_test_orig_pred = lr_classifier_origin.predict(X_test_scaled)

lr_classifier_transf = LogisticRegression(max_iter=1000)
lr_classifier_transf.fit(X_train_transf_scaled, y_train_transf,
                         sample_weight=w_train_transf)
y_test_transf_pred = lr_classifier_transf.predict(X_test_scaled)

In [42]:
# Evaluate models
print("MODEL EVALUATION")

print("MODEL (No Reweighing)")
print(f"Accuracy: {accuracy_score(y_test, y_test_orig_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_test_orig_pred, target_names=['Low Income', 'High Income']))

# Fairness metric for original model
dataset_orig_test_pred = dataset_orig_test.copy()
dataset_orig_test_pred.labels = y_test_orig_pred
metric_orig_test = ClassificationMetric(dataset_orig_test, dataset_orig_test_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
print(f"Statistical Parity Difference: {metric_orig_test.statistical_parity_difference():.4f}")

print("="*20)
print("MODEL (With Reweighing)")
print(f"Accuracy: {accuracy_score(y_test, y_test_transf_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_test_transf_pred, target_names=['Low Income', 'High Income']))

# Fairness metric for transformed model
dataset_transf_test_pred = dataset_orig_test.copy()
dataset_transf_test_pred.labels = y_test_transf_pred
metric_transf_test = ClassificationMetric(dataset_orig_test, dataset_transf_test_pred,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)
print(f"Statistical Parity Difference: {metric_transf_test.statistical_parity_difference():.4f}")
#TODO : SPD value (with reweighing) non-zero, why ?
#TODO : add other fairness metrics : balanced accuracy, disparate impact, average odds difference, (others, maybe?)



MODEL EVALUATION
MODEL (No Reweighing)
Accuracy: 0.8536
Classification Report:
              precision    recall  f1-score   support

  Low Income       0.88      0.93      0.90      5079
 High Income       0.75      0.63      0.68      1705

    accuracy                           0.85      6784
   macro avg       0.81      0.78      0.79      6784
weighted avg       0.85      0.85      0.85      6784

Statistical Parity Difference: -0.3649
MODEL (With Reweighing)
Accuracy: 0.8325
Classification Report:
              precision    recall  f1-score   support

  Low Income       0.85      0.95      0.89      5079
 High Income       0.76      0.49      0.60      1705

    accuracy                           0.83      6784
   macro avg       0.80      0.72      0.75      6784
weighted avg       0.82      0.83      0.82      6784

Statistical Parity Difference: -0.1441


In [41]:
print("COMPARISON")
print(f"Original Model Accuracy:    {accuracy_score(y_test, y_test_orig_pred):.4f}")
print(f"Transformed Model Accuracy: {accuracy_score(y_test, y_test_transf_pred):.4f}")
print(f"Accuracy Difference:        {accuracy_score(y_test, y_test_transf_pred) - accuracy_score(y_test, y_test_orig_pred):.4f}")
print(f"Original Model SPD:         {metric_orig_test.statistical_parity_difference():.4f}")
print(f"Transformed Model SPD:      {metric_transf_test.statistical_parity_difference():.4f}")
print(f"SPD Improvement:            {abs(metric_transf_test.statistical_parity_difference()) - abs(metric_orig_test.statistical_parity_difference()):.4f}")
print("Note: Statistical Parity Difference closer to 0 indicates better fairness")

COMPARISON
Original Model Accuracy:    0.8536
Transformed Model Accuracy: 0.8325
Accuracy Difference:        -0.0211
Original Model SPD:         -0.3649
Transformed Model SPD:      -0.1441
SPD Improvement:            -0.2208
Note: Statistical Parity Difference closer to 0 indicates better fairness
